In [60]:
import pandas as pd
import os
import ast

In [61]:
os.getcwd()

'c:\\HRW\\NLP\\projektabeit_nlp_2\\analyse_phase_01'

In [62]:
text_path = os.path.join('data', 'choc_recipes.txt')
# Using readlines()
file1 = open(text_path, 'r')
Lines = file1.readlines()

names = []
ingredients = []
steps = []

count = 0
# Strips the newline character
for line in Lines:
    pieces = line.split('\t')
    names.append(pieces[0])
    ingredients.append(pieces[1])
    steps.append(pieces[2])

In [63]:
print(f'Name count: {len(names)}')
print(f'Ingredient count: {len(ingredients)}')
print(f'Step count: {len(steps)}')

Name count: 7404
Ingredient count: 7404
Step count: 7404


In [ ]:
steps

In [95]:
ingredients_list_list = []
for i_string in ingredients:
    i = i_string.replace(', ', '*')
    i = i.replace(' ', '_')
    i = i.replace('*', ', ')

    ingredients_list_list.append(i)
ingredients_list_list

['butter, sugar, vanilla, eggs, all-purpose_flour, baking_cocoa, baking_powder, salt, miniature_peppermint_patties',
 'unsweetened_chocolate_squares, butter, sugar, eggs, flour, vanilla, nuts',
 'flour, sugar, cocoa, baking_powder, salt, milk, vegetable_oil, vanilla, brown_sugar, hot_water',
 'flour, brown_sugar, butter, pecans, semi-sweet_chocolate_chips',
 'flour, sugar, cocoa, baking_soda, salt, vegetable_oil, vinegar, vanilla, water',
 'bittersweet_chocolate, unsalted_butter, eggs, granulated_sugar, unsweetened_cocoa_powder, vanilla_extract, brewed_espresso, kosher_salt, all-purpose_flour',
 'flour, sugar, baking_soda, cocoa_powder, salt, mayonnaise, water, vanilla, instant_chocolate_pudding_mix, cool_whip, milk',
 'strong_coffee, chocolate_ice_cream, chocolate_syrup, sugar, ice, whipped_cream',
 "brownie_mix, water, vegetable_oil, eggs, salted_peanuts, reese's_peanut_butter_cups, milk_chocolate_chips, peanut_butter, butter, puffed_rice_cereal",
 'boiling_water, orange_gelatin, ice

In [96]:
ingredients = ingredients_list_list

In [181]:
import numpy as np
import tensorflow as tf

name_input = np.array(names)
ingredient_input = np.array(ingredients)
step_output = np.array(steps)

ingredient_dataset = tf.data.Dataset.from_tensor_slices(ingredient_input)
name_dataset =  tf.data.Dataset.from_tensor_slices(name_input)
step_dataset =  tf.data.Dataset.from_tensor_slices(step_output)
#dataset_12 = tf.data.Dataset.from_tensor_slices((name_input, ingredient_input))
#dataset_label = tf.data.Dataset.from_tensor_slices(step_output)

#dataset = tf.data.Dataset.zip((dataset_12, dataset_label)).batch(64)
#model.fit(dataset, epochs=10, steps_per_epoch=4)

In [182]:
import tensorflow as tf

max_features = 10000
max_len = 400


step_vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

#train_raw.map(lambda context, target: context)
step_vectorize_layer.adapt(step_dataset)

In [100]:
import tensorflow as tf

max_features = 2000
max_len = 4


ingredient_vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

#train_raw.map(lambda context, target: context)
ingredient_vectorize_layer.adapt(ingredient_dataset)

In [103]:
import tensorflow as tf

max_features = 5000
max_len = 4


name_vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

#train_raw.map(lambda context, target: context)
name_vectorize_layer.adapt(name_dataset)

In [104]:
name_vectorize_layer.get_vocabulary()

['',
 '[UNK]',
 'chocolate',
 'cake',
 'cookies',
 'chip',
 's',
 'brownies',
 'fudge',
 'butter',
 'peanut',
 'bars',
 'cream',
 'pie',
 'with',
 'and',
 'white',
 'hot',
 'pudding',
 'brownie',
 'muffins',
 'easy',
 'no',
 'cheesecake',
 'oatmeal',
 'banana',
 'double',
 'mocha',
 'cocoa',
 'cookie',
 'cupcakes',
 'caramel',
 'bread',
 'the',
 'free',
 'cherry',
 'low',
 'bake',
 'orange',
 'mousse',
 'frosting',
 'coffee',
 'ice',
 'fat',
 'squares',
 'sauce',
 'coconut',
 'almond',
 'best',
 'dark',
 'mix',
 'dessert',
 'raspberry',
 'mint',
 'toffee',
 'chewy',
 'cheese',
 'pecan',
 'vegan',
 'for',
 'black',
 'a',
 'rich',
 'nut',
 'truffles',
 'balls',
 'fudgy',
 'candy',
 'layer',
 'mexican',
 'triple',
 'or',
 'biscotti',
 'walnut',
 'of',
 'chunk',
 'bar',
 'cranberry',
 'cinnamon',
 'in',
 'decadent',
 'bundt',
 'kahlua',
 'milk',
 'creamy',
 'cakes',
 'marshmallow',
 'espresso',
 'sugar',
 'oreo',
 'torte',
 'pumpkin',
 'gluten',
 'strawberry',
 'one',
 'icing',
 'mini',
 '

In [146]:
from keras.layers import Input, Dense, concatenate, Embedding, ReLU, Dropout, Bidirectional, GRU, Add, Concatenate
from keras import Model

name_vocab_size = 256
calorie_vocab_size = 3
ingredient_vocab_size = 100

name_input = Input(name_vocab_size)
ingredient_input = Input(ingredient_vocab_size)

name_encoding = Embedding(name_vocab_size, 256, input_length=256)(name_input)
name_encoding = Bidirectional(GRU(128, return_sequences=True))(name_encoding)
#name_encoding = Model(inputs=name_input, outputs=name_encoding)

ingredient_encoding = Embedding(ingredient_vocab_size, 100, input_length=4)(ingredient_input)
ingredient_encoding = Dense(256)(ingredient_input)
ingredient_encoding = ReLU()(ingredient_encoding)
ingredient_encoding = Dropout(0.2)(ingredient_encoding)
#ingredient_encoding = Model(inputs= ingredient_input, outputs= ingredient_encoding)

added = Add()([name_encoding, ingredient_encoding])
out = Dense(128)(added)
#encoder = Model(inputs=[name_input, ingredient_input], outputs= out)
#combined = concatenate([name_encoding.output, calorie_encoding.output, ingredient_encoding.output])

In [110]:
#encoder = Model(inputs=[name_vectorize_layer, ingredient_vectorize_layer], outputs= out)

In [174]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, name_processor, ingredient_processor, units):
    super(Encoder, self).__init__()
    
    self.name_processor = name_processor
    self.ingredient_processor = ingredient_processor

    self.name_vocab_size = name_processor.vocabulary_size()
    self.ingredient_vocab_size = ingredient_processor.vocabulary_size()
    
    self.units = units

    # The embedding layer converts tokens to vectors
    self.name_embedding = tf.keras.layers.Embedding(self.name_vocab_size, units, mask_zero=True)
    self.ingredient_embedding = tf.keras.layers.Embedding(self.ingredient_vocab_size, units, mask_zero=True)

    self.concat = Concatenate()

    self.embedding = tf.keras.layers.Embedding((self.name_vocab_size + self.ingredient_vocab_size), units, mask_zero=True)

    # The RNN layer processes those vectors sequentially.
    self.rnn = tf.keras.layers.Bidirectional(
        merge_mode='sum',
        layer=tf.keras.layers.GRU(units,
                            # Return the sequence and state
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform'))

  def call(self, ingredients, names):

    # 2. The embedding layer looks up the embedding vector for each token.
    x1 = self.name_embedding(ingredients)
    x2 = self.ingredient_embedding(names)

    x = self.concat([x1, x2])
    x = self.embedding(x)
   
    # 3. The GRU processes the sequence of embeddings.
    x = self.rnn(x)
  
    # 4. Returns the new sequence of embeddings.
    return x

  def convert_input(self, ingredients, names):
    ingredients = tf.convert_to_tensor(ingredients)
    names = tf.convert_to_tensor(names)
    
    names = self.name_processor(names)#.to_tensor()
    ingredients = self.ingredient_processor(ingredients)#.to_tensor()
    
    context = self(ingredients, names)
    return context

In [175]:
UNITS = 256

# Encode the input sequence.
encoder = Encoder(name_vectorize_layer, ingredient_vectorize_layer, UNITS)



In [202]:
for (ex_name_tok) in name_dataset.take(1):
  print(ex_name_tok.numpy()) 

b'get the sensation  brownies'


In [203]:
for (ex_ingredient_tok) in ingredient_dataset.take(1):
  print(ex_ingredient_tok.numpy()) 

b'butter, sugar, vanilla, eggs, all-purpose_flour, baking_cocoa, baking_powder, salt, miniature_peppermint_patties'


In [204]:
for (ex_tar_in) in step_dataset.take(1):
  print(ex_tar_in.numpy()) 

b'preheat oven to 350 degrees <STEP> grease 13 x 9 baking pan <STEP> in a large bowl , whisk together butter , sugar and vanilla <STEP> add eggs and stir until well combined <STEP> combine dry ingredients and blend well with wet ingredients <STEP> reserve 2 cups of batter and set aside <STEP> spread remaining batter in prepared pan <STEP> arrange peppermint patties in a single layer over batter , about 1 / 2 inch apart <STEP> carefully spread reserved 2 cups of batter on top <STEP> bake for 50- 55 minutes or until brownies begin to pull away from sides of pan <STEP> cool completely on wire rack and cut into squares\n'


In [184]:
sample_name = name_dataset.take(1)
sample_ingredient = ingredient_dataset.take(1)
#ex_context = encoder(ex_ingredient_tok, ex_name_tok)
ex_context = encoder.convert_input(ex_ingredient_tok, ex_name_tok)

In [179]:
class CrossAttention(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=1, **kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

  def call(self, x, context):


    attn_output, attn_scores = self.mha(
        query=x,
        value=context,
        return_attention_scores=True)


    # Cache the attention scores for plotting later.
    attn_scores = tf.reduce_mean(attn_scores, axis=1)
    self.last_attention_weights = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [180]:
class Decoder(tf.keras.layers.Layer):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, text_processor, units):
    super(Decoder, self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.word_to_id = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]')
    self.id_to_word = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]',
        invert=True)
    self.start_token = self.word_to_id('[START]')
    self.end_token = self.word_to_id('[END]')

    self.units = units


    # 1. The embedding layer converts token IDs to vectors
    self.embedding = tf.keras.layers.Embedding(self.vocab_size,
                                               units, mask_zero=True)

    # 2. The RNN keeps track of what's been generated so far.
    self.rnn = tf.keras.layers.GRU(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

    # 3. The RNN output will be the query for the attention layer.
    self.attention = CrossAttention(units)

    # 4. This fully connected layer produces the logits for each
    # output token.
    self.output_layer = tf.keras.layers.Dense(self.vocab_size)

In [197]:
@Decoder.add_method
def call(self,
         context, x,
         state=None,
         return_state=False):  
  # 1. Lookup the embeddings
  x = self.embedding(x)

  # 2. Process the target sequence.
  x, state = self.rnn(x, initial_state=state)
  # 3. Use the RNN output as the query for the attention over the context.
  x = self.attention(x, context)
  self.last_attention_weights = self.attention.last_attention_weights

  # Step 4. Generate logit predictions for the next token.
  logits = self.output_layer(x)

  if return_state:
    return logits, state
  else:
    return logits

In [200]:
decoder = Decoder(step_vectorize_layer, UNITS)

In [205]:
logits = decoder(ex_context, ex_tar_in)

print(f'encoder output shape: (batch, s, units) {ex_context.shape}')
print(f'input target tokens shape: (batch, t) {ex_tar_in.shape}')
print(f'logits shape shape: (batch, target_vocabulary_size) {logits.shape}')

UnimplementedError: Exception encountered when calling layer 'embedding_33' (type Embedding).

{{function_node __wrapped__Cast_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cast string to int32 is not supported [Op:Cast]

Call arguments received by layer 'embedding_33' (type Embedding):
  • inputs=tf.Tensor(shape=(), dtype=string)

In [ ]:
@Decoder.add_method
def get_initial_state(self, context):
  batch_size = tf.shape(context)[0]
  start_tokens = tf.fill([batch_size, 1], self.start_token)
  done = tf.zeros([batch_size, 1], dtype=tf.bool)
  embedded = self.embedding(start_tokens)
  return start_tokens, done, self.rnn.get_initial_state(embedded)[0]

In [ ]:
@Decoder.add_method
def tokens_to_text(self, tokens):
  words = self.id_to_word(tokens)
  result = tf.strings.reduce_join(words, axis=-1, separator=' ')
  result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
  result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
  return result

In [ ]:
@Decoder.add_method
def get_next_token(self, context, next_token, done, state, temperature = 0.0):
  logits, state = self(
    context, next_token,
    state = state,
    return_state=True) 

  if temperature == 0.0:
    next_token = tf.argmax(logits, axis=-1)
  else:
    logits = logits[:, -1, :]/temperature
    next_token = tf.random.categorical(logits, num_samples=1)

  # If a sequence produces an `end_token`, set it `done`
  done = done | (next_token == self.end_token)
  # Once a sequence is done it only produces 0-padding.
  next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)

  return next_token, done, state

In [206]:
# Setup the loop variables.
next_token, done, state = decoder.get_initial_state(ex_context)
tokens = []

for n in range(10):
  # Run one step.
  next_token, done, state = decoder.get_next_token(
      ex_context, next_token, done, state, temperature=1.0)
  # Add the token to the output.
  tokens.append(next_token)

# Stack all the tokens together.
tokens = tf.concat(tokens, axis=-1) # (batch, t)

# Convert the tokens back to a a string
result = decoder.tokens_to_text(tokens)
result[:3].numpy()

array([b'ala tightly scoring unhealthy 4cmdeep momentarily dove forget decors coes',
       b'4quart chops harder saute cookiesi 365 rope equals cornstarchjuice turntable',
       b'prepare almondjoyee yellow puffs holly 3cm plaace ina supposedly squiggles'],
      dtype=object)